### ![](../../img/installation-ico.png) Installation

In [ ]:
pip install langchain-cli langserve

### ![](../../img/parametrage-ico.png) Paramétrages

In [ ]:
# Au lieu de tout recréer une fois le CLI installer utiliser la commande suivante dans une invite de commande pour initier l'app
langchain app new school-app

# Et ajouter les chains ou runnables souhaitées

In [ ]:
# app/server.py

# Ajout d'une route pour un LLM simple
add_routes(app, VertexAI(model_name="gemini-pro", temperature=0.1), path="/chat")


# Ajout d'une route pour une chaîne
prompt = PromptTemplate.from_template(
    "Explique moi simplement le concept suivant : {topic}"
)
llm = VertexAI(model_name="gemini-pro", temperature=0.1)
output_parser = StrOutputParser()
chain = prompt | llm | output_parser
add_routes(app, chain, path="/chain")

### ![](../../img/jouer-ico.png) Exécution(s)

In [ ]:
langchain serve --port=8100

In [ ]:
docker build . -t school-app

### Commandes d'appels

#### Version Remote Runnable

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langserve import RemoteRunnable

remoteLLM = RemoteRunnable("http://localhost:8100/chat/")

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a highly educated person who loves to use big words. "
            + "You are also concise. Never answer in more than three sentences.",
        ),
        ("human", "Tell me about {topic}"),
    ]
)

chain = prompt | remoteLLM

for chunk in chain.stream({"topic": "quantum computing"}):
    print(chunk, end="", flush=True)

In [ ]:
remoteChain = RemoteRunnable("http://localhost:8100/chain/")

remoteChain.invoke({"topic": "ordinateur quantique"})

#### Version CURL (ou équivalent)


```shell
curl --location --request POST 'http://localhost:8100/chat/stream' \
--header 'Content-Type: application/json' \
--data '{
    "input": "What is quantum computing ?"
}'
```

```shell
curl --location --request POST 'http://localhost:8100/chain/invoke' \
--header 'Content-Type: application/json' \
--data '{
    "input": {
        "topic": "LangServe (de LangChaine)"
    }
}'
```